In [1]:
import pandas as pd
import numpy as np


df_train = pd.read_csv("train.csv")
df_train

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54268,54268,BMW,X6 xDrive50i,2017,29000,Gasoline,445.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Brown,None reported,Yes,29000
54269,54269,Audi,A4 2.0T Premium,2015,94634,E85 Flex Fuel,220.0HP 2.0L 4 Cylinder Engine Flex Fuel Capab...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,6500
54270,54270,Porsche,Cayenne S,2013,40989,Gasoline,420.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Black,At least 1 accident or damage reported,Yes,18950
54271,54271,Porsche,911 Carrera 4 GTS,2023,1518,Gasoline,4.0L H6 24V GDI DOHC,8-Speed Automatic with Auto-Shift,Beige,Brown,None reported,Yes,194965


In [2]:
df_test = pd.read_csv("test.csv")

In [3]:
df_train['horsepower'] = df_train['engine'].str.extract(r'(\d+\.?\d*)HP').astype(float)
df_train['engine_volume'] = df_train['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)

df_test['horsepower'] = df_test['engine'].str.extract(r'(\d+\.?\d*)HP').astype(float)
df_test['engine_volume'] = df_test['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)

In [4]:
mean_price_per_brand = df_train.groupby('brand')['price'].mean()

df_train['Brand_Encoded'] = df_train['brand'].map(mean_price_per_brand)
df_test['Brand_Encoded'] = df_test['brand'].map(mean_price_per_brand)

In [5]:
X_train = df_train.copy()
y_train = X_train.pop('price')

X_test = df_test.copy()

X_train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,horsepower,engine_volume,Brand_Encoded
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,375.0,3.5,38154.063227
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,300.0,3.0,40276.029448
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,300.0,4.2,34840.403933
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,335.0,3.0,40276.029448
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,200.0,3.8,17526.060403


In [6]:
X_train.columns

Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'horsepower', 'engine_volume', 'Brand_Encoded'],
      dtype='object')

In [7]:
#select low cardinality columns
low_cardinality_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and X_train[cname].dtype == "object"]

#select numeric columns
numeric_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

new_features = low_cardinality_cols + numeric_cols

#lets use the selected columns only 
X_train = X_train[new_features]
X_train.columns

Index(['fuel_type', 'accident', 'clean_title', 'id', 'model_year', 'milage',
       'horsepower', 'engine_volume', 'Brand_Encoded'],
      dtype='object')

In [8]:
X_train.accident.unique()

array(['None reported', 'At least 1 accident or damage reported'],
      dtype=object)

In [9]:
from sklearn.preprocessing import OneHotEncoder

# Define a function to OHE encode
def OHE_encoder(OHE_X):
    encoder = OneHotEncoder(sparse_output=False)
    OneHotEncoded  = encoder.fit_transform(OHE_X[low_cardinality_cols])
    OHE_X = pd.DataFrame(OneHotEncoded, columns= encoder.get_feature_names_out(low_cardinality_cols))
    return OHE_X

# Encode Train and Test data
X_train_OHE = OHE_encoder(X_train)
X_train_OHE = pd.concat([X_train[numeric_cols], X_train_OHE], axis=1)
X_test_OHE = OHE_encoder(X_test)
X_test_OHE = pd.concat([X_test[numeric_cols], X_test_OHE], axis=1)


In [10]:
X_train_OHE

,id,model_year,milage,horsepower,engine_volume,Brand_Encoded,fuel_type_Diesel,fuel_type_E85 Flex Fuel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_Plug-In Hybrid,fuel_type_not supported,fuel_type_–,accident_At least 1 accident or damage reported,accident_None reported,clean_title_Yes
0,0,2018,74349,375.0,3.5,38154.063227,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,1,2007,80000,300.0,3.0,40276.029448,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,2,2009,91491,300.0,4.2,34840.403933,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,3,2022,2437,335.0,3.0,40276.029448,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
4,4,2001,111000,200.0,3.8,17526.060403,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54268,54268,2017,29000,445.0,4.4,40276.029448,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
54269,54269,2015,94634,220.0,2.0,37091.368241,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
54270,54270,2013,40989,420.0,3.6,63742.154930,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
54271,54271,2023,1518,NaN,4.0,63742.154930,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [41]:
from sklearn.model_selection import train_test_split

train_X, valid_X, train_y, valid_y = train_test_split(X_train_OHE,y_train, test_size= 0.2)

In [49]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

model = XGBRegressor(n_estimators=1000, learning_rate=0.0001, max_depth = 30, random_state=0)
model.fit(train_X, train_y, early_stopping_rounds= 19, eval_set=[(valid_X, valid_y)])

prediction_01 = model.predict(valid_X)

print("Mean Absolute Error: ", mean_absolute_error(prediction_01, valid_y))

c:\Users\rohan\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-rmse:70149.85627
[1]	validation_0-rmse:70149.01367
[2]	validation_0-rmse:70148.17190
[3]	validation_0-rmse:70147.32931
[4]	validation_0-rmse:70146.48843
[5]	validation_0-rmse:70145.64844
[6]	validation_0-rmse:70144.80812
[7]	validation_0-rmse:70143.96823
[8]	validation_0-rmse:70143.12823
[9]	validation_0-rmse:70142.28883
[10]	validation_0-rmse:70141.44855
[11]	validation_0-rmse:70140.61120
[12]	validation_0-rmse:70139.77355
[13]	validation_0-rmse:70138.93466
[14]	validation_0-rmse:70138.09656
[15]	validation_0-rmse:70137.25966
[16]	validation_0-rmse:70136.42321
[17]	validation_0-rmse:70135.58698
[18]	validation_0-rmse:70134.74892
[19]	validation_0-rmse:70133.91216
[20]	validation_0-rmse:70133.07491
[21]	validation_0-rmse:70132.23985
[22]	validation_0-rmse:70131.40306
[23]	validation_0-rmse:70130.56937
[24]	validation_0-rmse:70129.73314
[25]	validation_0-rmse:70128.89892
[26]	validation_0-rmse:70128.06493
[27]	validation_0-rmse:70127.23100
[28]	validation_0-rmse:70126.3

In [51]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

param_grid = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 10, 20]
}

# Create the model
model = XGBRegressor(random_state=0)

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           scoring='neg_mean_absolute_error', cv=5, verbose=2)

# Fit the GridSearchCV object to the data
grid_search.fit(train_X, train_y)

# Get the best parameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Print the best parameters
print("Best parameters found: ", best_params)

# Make predictions with the best model
prediction_01 = best_model.predict(valid_X)

# Evaluate the model
print("Mean Absolute Error: ", mean_absolute_error(prediction_01, valid_y))


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=   0.5s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=   0.6s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=   0.5s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=   0.5s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=   0.5s
[CV] END .learning_rate=0.01, max_depth=3, n_estimators=1000; total time=   1.1s
[CV] END .learning_rate=0.01, max_depth=3, n_es

In [46]:
print(prediction_01)

[43593.49  52976.094 84133.79  ... 58949.277 50412.65  23486.428]


In [44]:
print(y_train)

0         11000
1          8250
2         15000
3         63500
4          7850
          ...  
54268     29000
54269      6500
54270     18950
54271    194965
54272     37499
Name: price, Length: 54273, dtype: int64


In [15]:
X_test_OHE['price'] = model.predict(X_test_OHE)

In [16]:
X_test_OHE

,id,model_year,milage,horsepower,engine_volume,Brand_Encoded,fuel_type_Diesel,fuel_type_E85 Flex Fuel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_Plug-In Hybrid,fuel_type_not supported,fuel_type_–,accident_At least 1 accident or damage reported,accident_None reported,clean_title_Yes,price
0,54273,2014,73000,302.0,3.5,47094.039119,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,25216.361328
1,54274,2015,128032,275.0,3.5,30685.441294,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,20513.724609
2,54275,2015,51983,241.0,2.0,47094.039119,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,31403.748047
3,54276,2018,29500,518.0,5.0,47714.265664,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,62471.785156
4,54277,2020,90000,335.0,3.0,40276.029448,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,39793.707031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36178,90451,2019,4500,420.0,6.2,41570.947955,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,70765.070312
36179,90452,2004,185000,295.0,5.3,44751.109855,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,16595.558594
36180,90453,2011,116000,132.0,1.8,29079.996112,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,13378.248047
36181,90454,2019,39000,450.0,3.5,27733.617992,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,51692.394531


In [17]:
X_test_OHE[['id', 'price']].to_csv("submission_5.csv", index=False) 